In [2]:
from fine_tuned import AdaptedModel
from cnn_lstm_1d import CNNLSTM

In [3]:
pretrained = CNNLSTM()
model = AdaptedModel(pretrained)
model.load('weights/adapted.pth')

/Users/supremegg/Documents/GitHub/surgical-hand-tremor/model/fine_tuned.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map_location=s

RuntimeError: Error(s) in loading state_dict for AdaptedModel:
	Missing key(s) in state_dict: "adapter.0.running_mean", "adapter.0.running_var", "adapter.1.weight", "adapter.1.bias", "adapter.4.weight", "adapter.4.bias". 
	Unexpected key(s) in state_dict: "adapter.2.weight", "adapter.2.bias". 
	size mismatch for adapter.0.weight: copying a param with shape torch.Size([64, 32]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for adapter.0.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([32]).

In [ ]:
# Now run a classification test
import pandas as pd

df = pd.read_csv('data/tremor_data.csv')
aX, aY, aZ = df['aX'], df['aY'], df['aZ']
result = df['Result']

# Now we need to convert the data into the format that the model expects
import numpy as np
# The model expects : (batch, window len, 3)
# We will use a window length of 100
window_len = 100
data = np.zeros((len(aX) - window_len, window_len, 3))
for i in range(len(aX) - window_len):
    data[i, :, 0] = aX[i:i + window_len]
    data[i, :, 1] = aY[i:i + window_len]
    data[i, :, 2] = aZ[i:i + window_len]

# Now we can run the model
import torch
data = torch.tensor(data, dtype=torch.float32)
prediction = model(data)

# Now we can compare the prediction to the actual result
import matplotlib.pyplot as plt
plt.plot(prediction.detach().numpy())
plt.plot(result[window_len:])
plt.show()